In [1]:
import cv2
import pyttsx3
import numpy as np

pedestrian_cascade = cv2.CascadeClassifier('fullbody.xml')
fullbody_cascade = cv2.CascadeClassifier('upperbody.xml')

hand_cascade = cv2.CascadeClassifier('hand.xml')

video_source = "people.mp4"  
cap = cv2.VideoCapture(video_source)

people_count = 0
group_count = 0
group_threshold = 10  
message = ""
crowd_message = "The area is crowded."  
normal_message = "The area is in normal state."  

engine = pyttsx3.init()

crowd_message_count = 0
normal_message_count = 0

lower_degrees = 90  
upper_degrees = 150  

def speak_message(message):
    engine.say(message)
    engine.runAndWait()

def detect_crime(frame):
    return False

while True:
    ret, frame = cap.read()

    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    pedestrians = pedestrian_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    fullbodies = fullbody_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    hands = hand_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    all_people = list(pedestrians) + list(fullbodies)
    all_hands = list(hands)

    frame_group_count = 0

    for (x, y, w, h) in all_people:
        if w * h > 1000:  
            if frame_group_count == 0:
                group_count += 1
            frame_group_count += 1
            color = (0, 0, 255) if frame_group_count > group_threshold else (0, 255, 0)
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

    for (x, y, w, h) in all_hands:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    hand_posture_detected = False
    for (x, y, w, h) in all_hands:
        hand_center_x = x + w // 2
        hand_tip_x = x + w
        hand_center_y = y + h // 2
        hand_tip_y = y
        angle_degrees = np.degrees(np.arctan2(hand_center_y - hand_tip_y, hand_tip_x - hand_center_x))

        if lower_degrees < angle_degrees < upper_degrees:
            hand_posture_detected = True
            break

    if hand_posture_detected:
        message = "Suspicious Activity Detected (Raised Hand)"
        if 'speech' not in message.lower():
            speak_message(message)

    if detect_crime(frame):
        message = "Suspicious Activity Detected"
        if 'speech' not in message.lower():
            speak_message(message)

    people_count = len(all_people)
    
    if people_count > 20 and crowd_message_count < 1:
        message = "Crowd Formed"
        if 'speech' not in message.lower():
            speak_message(crowd_message)
            crowd_message_count += 1
        normal_message_count = 0
    elif people_count <= 20 and normal_message_count < 1:
        message = "Normal State"
        if 'speech' not in message.lower():
            speak_message(normal_message)
            normal_message_count += 1
        crowd_message_count = 0
    else:
        message = ""

    cv2.putText(frame, f"People Count: {people_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(frame, message, (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("People Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
